In [1]:
import os

In [2]:
%pwd

'/media/essam/01DBE776362F5370/Courses/End-to-End MLOps Projects/Krish/End-to-end-ML-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/media/essam/01DBE776362F5370/Courses/End-to-End MLOps Projects/Krish/End-to-end-ML-Project-with-MLflow'

In [5]:
import dagshub
dagshub.init(repo_owner='EssamShenhab', repo_name='End-to-end-ML-Project-with-MLflow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as EssamShenhab

Initialized MLflow to track repo "EssamShenhab/End-to-end-ML-Project-with-MLflow"

Repository EssamShenhab/End-to-end-ML-Project-with-MLflow initialized!

🏃 View run whimsical-skink-954 at: https://dagshub.com/EssamShenhab/End-to-end-ML-Project-with-MLflow.mlflow/#/experiments/0/runs/700adbe15d214fd3a8988803f4f713e9
🧪 View experiment at: https://dagshub.com/EssamShenhab/End-to-end-ML-Project-with-MLflow.mlflow/#/experiments/0


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/EssamShenhab/End-to-end-ML-Project-with-MLflow.mlflow",
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models import infer_signature

In [10]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        """
        Initialize with a config object containing evaluation settings.
        """
        self.config = config

    def eval_metrics(self, actual, pred):
        """
        Compute evaluation metrics.
        """
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        """
        Evaluate model, log metrics and model to MLflow, and save metrics locally.
        """
        try:
            # Load test data and model
            test_data = pd.read_csv(self.config.test_data_path)
            model = joblib.load(self.config.model_path)

            test_x = test_data.drop(columns=[self.config.target_column])
            test_y = test_data[self.config.target_column].values  # Convert to 1D array

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                predicted_qualities = model.predict(test_x)

                # Evaluate
                rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

                # Save metrics locally
                scores = {"rmse": rmse, "mae": mae, "r2": r2}
                save_json(path=Path(self.config.metric_file_name), data=scores)

                # Log parameters and metrics
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

                # Log model (and register if not file-based)
                input_example = test_x.iloc[:1]
                signature = infer_signature(test_x, predicted_qualities)

                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(
                        model,
                        "model",
                        registered_model_name="ElasticnetModel",
                        input_example=input_example,
                        signature=signature
                    )
                else:
                    mlflow.sklearn.log_model(
                        model,
                        "model",
                        input_example=input_example,
                        signature=signature
                    )

        except Exception as e:
            raise RuntimeError(f"Model evaluation failed: {e}")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-06 01:47:40,210: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-06 01:47:40,211: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-06 01:47:40,214: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-06 01:47:40,215: INFO: common: created directory at: artifacts]
[2025-08-06 01:47:40,217: INFO: common: created directory at: artifacts/model_evaluation]


[2025-08-06 01:47:40,697: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/08/06 01:47:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run victorious-croc-964 at: https://dagshub.com/EssamShenhab/End-to-end-ML-Project-with-MLflow.mlflow/#/experiments/0/runs/5a7eeee1c0964ac8beab475b44e6dc2a
🧪 View experiment at: https://dagshub.com/EssamShenhab/End-to-end-ML-Project-with-MLflow.mlflow/#/experiments/0
